In [28]:
import pandas as pd
import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
import matplotlib.pyplot as plt

import tournament_new as tn
import create_database as cd
importlib.reload(tn)

import beck.beck_game
from importlib import reload
reload(beck.beck_game)
from beck.beck_game import BeckGame as Game
game = Game(4,9,4)

from arena import Arena
from mcts import MCTS
importlib.reload(tn)
game = Game(4,9,4)
all_p = pd.read_pickle(cd.DATABASE_LOC)

# res = tn.merge_res_to_base()
tournament_res = pd.read_pickle('/scratch/zz737/fiar/tournaments/ai_all_player_round_robin_base.pkl')

import argparse
import os
import shutil
import time
import random
import numpy as np
import math
import sys

import keras
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import metrics

sys.path.append('..')
from neural_net import NeuralNet
from utils import *

reload(beck.beck_nnet)
from beck.beck_nnet import OthelloNNet, NNetWrapper

import beck.beck_nnet_pvcorr as bnp
reload(bnp)

<module 'beck.beck_nnet_pvcorr' from '../classes/beck/beck_nnet_pvcorr.py'>

In [130]:
# load and organize data
from pickle import Pickler, Unpickler
weight_dir = '/scratch/zz737/fiar/tournaments/tournament_6/checkpoints_mcts100_cpuct2_id_0'
fn = 'checkpoint_8.pth.tar'
load_folder_file = (weight_dir,fn)

modelFile = os.path.join(load_folder_file[0], load_folder_file[1])
examplesFile = modelFile + ".examples"
with open(examplesFile, "rb") as f:
    trainExamplesHistory = Unpickler(f).load()


from random import shuffle
trainExamples = []
for e in trainExamplesHistory:
    trainExamples.extend(e)
shuffle(trainExamples)
N_trainexs = len(trainExamples)

# input_boards, target_pis, target_vs = list(zip(*trainExamples[:10]))
# input_boards = np.asarray(input_boards)
# target_pis = np.asarray(target_pis)
# target_vs = np.asarray(target_vs)

In [131]:
# create agent before training:

args = bnp.args
args.batch_size = 50
assert N_trainexs % args.batch_size ==0

pnet = NNetWrapper(game)
pnet.load_checkpoint(folder=weight_dir, filename=fn)

nnet = NNetWrapper(game,args=args)
nnet.load_checkpoint(folder=weight_dir, filename=fn)

reload(bnp)
nnet_pvcorr = bnp.NNetWrapper_pvcorr(game,args=args)
nnet_pvcorr.load_checkpoint(folder=weight_dir, filename=fn)

nnet_dict = {'p':pnet,'n':nnet,'n_pvcorr':nnet_pvcorr}

In [ ]:
# training
for pl, val in nnet_dict.items():
    if pl!='p':
        print(f'player {pl}')
        val.train(trainExamples)

player n
Epoch 1/10
561/561 [==============================] - 502s 894ms/step - loss: 2.4295 - pi_loss: 2.1169 - v_loss: 0.3126
Epoch 2/10
561/561 [==============================] - 500s 892ms/step - loss: 2.3755 - pi_loss: 2.0833 - v_loss: 0.2921
Epoch 3/10
561/561 [==============================] - 500s 891ms/step - loss: 2.3540 - pi_loss: 2.0709 - v_loss: 0.2832
Epoch 4/10
561/561 [==============================] - 501s 893ms/step - loss: 2.3445 - pi_loss: 2.0620 - v_loss: 0.2826
Epoch 5/10
561/561 [==============================] - 495s 882ms/step - loss: 2.3347 - pi_loss: 2.0560 - v_loss: 0.2787
Epoch 6/10
561/561 [==============================] - 493s 879ms/step - loss: 2.3255 - pi_loss: 2.0507 - v_loss: 0.2747
Epoch 7/10
561/561 [==============================] - 493s 879ms/step - loss: 2.3169 - pi_loss: 2.0452 - v_loss: 0.2717
Epoch 8/10
561/561 [==============================] - 492s 878ms/step - loss: 2.3172 - pi_loss: 2.0442 - v_loss: 0.2730
Epoch 9/10
561/561 [===========

In [119]:
# create mcts and agents 
n_mcts = 100
cpuct = 2
tree_args = dotdict({
'numMCTSSims': n_mcts,
'cpuct': cpuct,
})
tree_dict = {}
ai_dict = {}
for player,val_func in nnet_dict.items():
    tree_dict[player] = MCTS(game,val_func,tree_args)
    ai_dict[player] = lambda x: np.random.choice(np.arange(game.getActionSize()),p=tree_dict[player].getActionProb(x, temp=1/10))



In [122]:
participants = [['p','n'],['p','n_pvcorr'],['n','n_pvcorr']]
res_l = []
ngames = 2
for p1,p2 in participants:
    print(f'{p1} vs {p2}')
    arena = Arena(ai_dict[p1],ai_dict[p2],game,display=game.display,tree1=tree_dict[p1],tree2=tree_dict[p2])
    res=arena.playGames(ngames,verbose=True)
    res_l.append(res)
    print(f'{p1} vs {p2}')
    print(f'result {res}')
    
for p1,p2 in participants:
    print(f'{p1} vs {p2}')
    print(f'result {res}')

Arena.playGames (1):   0%|          | 0/1 [00:00<?, ?it/s]

p vs n
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O - X - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O O X - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- X - O O X - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  6 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |

Arena.playGames (2):   0%|          | 0/1 [00:00<?, ?it/s]

Game over: Turn  16 Result  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O - X - - - - - - |
1 |- X O O O X - - - |
2 |- - X - O O - - - |
3 |X X X X O O - - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - X O - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - X O - - - - - |
2 |- - - - O - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - X - - - - - - 

Arena.playGames (1):   0%|          | 0/1 [00:00<?, ?it/s]

...

Turn  15 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X X - O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Turn  16 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X X O O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Game over: Turn  16 Result  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X X O O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
result (2, 0, 0)
n vs n_pvcorr
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
------------

Arena.playGames (2):   0%|          | 0/1 [00:00<?, ?it/s]

Turn  15 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X X - O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Turn  16 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X X O O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Game over: Turn  16 Result  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X X O O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - 

Arena.playGames (2): 100%|██████████| 1/1 [00:27<00:00, 27.49s/it]

Turn  15 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X X - O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Turn  16 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X X O O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
Game over: Turn  16 Result  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X X O O O - - |
1 |- - X O X - - - - |
2 |- - X O O O X - - |
3 |- - O - - - - - - |
-----------------------
result (1, 1, 0)
